In [ ]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
import hook_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *
import csv

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-large")

with open('data/gpt2_large_spectrum.pkl', 'rb') as f:
    sample_df = pickle.load(f)

with open('data/gpt2_large_spectrum_bins.pkl', 'rb') as f:
    bin_edges = pickle.load(f)

def read_labels(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return {row[0]: row[1] for row in csv.reader(f)}
    return {}
read_labels('data/gp2_large_spectrum_labels.csv')

In [ ]:
grouped = sample_df.groupby('L31N892 Activation', as_index=False)['next_is_an'].agg(['sum', 'count'])
grouped['Next token is \' an\''] = grouped['sum'] / grouped['count']
grouped['Next token is not \' an\''] = 1 - grouped['Next token is \' an\'']

grouped_melted = grouped.melt(id_vars=['L31N892 Activation'], value_vars=['Next token is \' an\'', 'Next token is not \' an\''], 
                            var_name='status', value_name='Proportion of Labels')
fig = px.area(grouped_melted, x='L31N892 Activation', y='Proportion of Labels', color='status', line_shape='spline')
tickvals = np.arange(len(bin_edges) - 1)
ticktext = [f"{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}" for i in tickvals]

fig.update_layout(
    yaxis=dict(range=[0, 1]),
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ),
)
fig.show()